In [1]:
import torch
from torch import nn
import utils

In [2]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 通过is_grad_enabled来判断当前模式是训练模式还是预测模式
    if not torch.is_grad_enabled():
        # 预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。
            # 这里我们需要保持X的形状以便后面可以做广播运算
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下，用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 缩放和移位
    return Y, moving_mean.data, moving_var.data

In [3]:
class BatchNorm(nn.Module):
    # num_features：完全连接层的输出数量或卷积层的输出通道数。
    # num_dims：2表示完全连接层，4表示卷积层
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

In [4]:
# 使用批量规范化层的 LeNet
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), 
    BatchNorm(6, num_dims=4), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(6, 16, kernel_size=5), 
    BatchNorm(16, num_dims=4), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), 
    
    nn.Flatten(),
    nn.Linear(16*4*4, 120), 
    BatchNorm(120, num_dims=2), 
    nn.Sigmoid(),
    
    nn.Linear(120, 84), 
    BatchNorm(84, num_dims=2), 
    nn.Sigmoid(),
    
    nn.Linear(84, 10))

In [5]:
lr, num_epochs, batch_size = 1.0, 10, 256
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size)
utils.train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-06-01 00:46:45]
epoch: 1/10, loss 0.734, train acc 0.734, test acc 0.711
epoch: 2/10, loss 0.479, train acc 0.825, test acc 0.817
epoch: 3/10, loss 0.405, train acc 0.854, test acc 0.676
epoch: 4/10, loss 0.361, train acc 0.868, test acc 0.841
epoch: 5/10, loss 0.331, train acc 0.880, test acc 0.861
epoch: 6/10, loss 0.311, train acc 0.887, test acc 0.860
epoch: 7/10, loss 0.295, train acc 0.892, test acc 0.869
epoch: 8/10, loss 0.284, train acc 0.895, test acc 0.881
epoch: 9/10, loss 0.271, train acc 0.900, test acc 0.856
epoch: 10/10, loss 0.260, train acc 0.904, test acc 0.836
*** 24819.4 examples/sec on cuda:0 - [0:00:00:24], all: [0:00:01:00] ***


In [6]:
# 查看第一个批量规范化层中学到的拉伸参数gamma和偏移参数beta。
net[1].gamma.reshape((-1,)), net[1].beta.reshape((-1,))

(tensor([3.4609, 1.9792, 3.2465, 3.2853, 2.5997, 1.9066], device='cuda:0',
        grad_fn=<ViewBackward>),
 tensor([-1.7435, -1.1527, -2.0026, -3.5925,  2.2950, -2.0930], device='cuda:0',
        grad_fn=<ViewBackward>))

In [7]:
# 简明实现

net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), 
    nn.BatchNorm2d(6), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(6, 16, kernel_size=5), 
    nn.BatchNorm2d(16), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), 
    
    nn.Flatten(),
    nn.Linear(256, 120), 
    nn.BatchNorm1d(120), 
    nn.Sigmoid(),
    
    nn.Linear(120, 84), 
    nn.BatchNorm1d(84), 
    nn.Sigmoid(),
    
    nn.Linear(84, 10))

In [8]:
utils.train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-06-01 00:52:20]
epoch: 1/10, loss 0.727, train acc 0.736, test acc 0.767
epoch: 2/10, loss 0.475, train acc 0.828, test acc 0.774
epoch: 3/10, loss 0.396, train acc 0.855, test acc 0.835
epoch: 4/10, loss 0.359, train acc 0.869, test acc 0.822
epoch: 5/10, loss 0.334, train acc 0.878, test acc 0.553
epoch: 6/10, loss 0.316, train acc 0.884, test acc 0.797
epoch: 7/10, loss 0.300, train acc 0.891, test acc 0.695
epoch: 8/10, loss 0.287, train acc 0.894, test acc 0.834
epoch: 9/10, loss 0.276, train acc 0.898, test acc 0.800
epoch: 10/10, loss 0.269, train acc 0.901, test acc 0.872
*** 42484.4 examples/sec on cuda:0 - [0:00:00:14], all: [0:00:00:49] ***
